In [1]:
import gurobipy as gp
from gurobipy import GRB

# 1. Données du problème (Xavier vs Yvonne)
# 
matieres = ["Anatomie", "Biologie", "Chirurgie", "Diagnostic", "Epidemiologie", "Forensic Pathology", "Génétique"]
poids = [8, 7, 7, 6, 6, 5, 6]

# Notes des candidats 
notes_x = [85, 81, 71, 69, 75, 81, 88] # Xavier
notes_y = [81, 81, 75, 63, 67, 88, 95] # Yvonne

# 2. Calcul des contributions pondérées
# La contribution est : poids * (note_x - note_y)
contributions = []
for i in range(len(matieres)):
    delta = notes_x[i] - notes_y[i]
    contrib = poids[i] * delta
    contributions.append(contrib)

print("Calcul des contributions par matière :")
for m, c in zip(matieres, contributions):
    print(f"{m}: {c:+}")

# Vérification avec le tableau de la page 2[cite: 30]:
# A (Anatomie): +32, C (Chirurgie): -28, etc.

Calcul des contributions par matière :
Anatomie: +32
Biologie: +0
Chirurgie: -28
Diagnostic: +36
Epidemiologie: +48
Forensic Pathology: -35
Génétique: -42


In [2]:
# 3. Identification des ensembles Pros (P) et Cons (C)
P_indices = [i for i, c in enumerate(contributions) if c > 0] # Indices des matières "Pour"
C_indices = [j for j, c in enumerate(contributions) if c < 0] # Indices des matières "Contre"

print(f"\nEnsemble Pros (Indices) : {P_indices} -> {[matieres[i] for i in P_indices]}")
print(f"Ensemble Cons (Indices) : {C_indices} -> {[matieres[j] for j in C_indices]}")

# Calcul de la matrice de validité des trade-offs (1-1)
# Un trade-off (i, j) est valide si contribution[i] + contribution[j] > 0 [cite: 31]
valid_pairs = []
for i in P_indices:
    for j in C_indices:
        if contributions[i] + contributions[j] > 0:
            valid_pairs.append((i, j))

print(f"\nNombre de paires (1-1) valides identifiées : {len(valid_pairs)}")


Ensemble Pros (Indices) : [0, 3, 4] -> ['Anatomie', 'Diagnostic', 'Epidemiologie']
Ensemble Cons (Indices) : [2, 5, 6] -> ['Chirurgie', 'Forensic Pathology', 'Génétique']

Nombre de paires (1-1) valides identifiées : 6


In [3]:
# 4. Création du modèle d'optimisation
m = gp.Model("Explication_X_sup_Y_Type_1_1")

# Variables de décision : x[i,j] = 1 si on utilise la matière i pour compenser la matière j
# On ne crée les variables QUE pour les paires valides pour réduire l'espace de recherche
x = {}
for i, j in valid_pairs:
    x[i, j] = m.addVar(vtype=GRB.BINARY, name=f"pair_{matieres[i]}_{matieres[j]}")

# Objectif : Maximiser le nombre de critères "Contre" expliqués (couverts)
# Max sum(x_ij)
m.setObjective(gp.quicksum(x[i, j] for i, j in valid_pairs), GRB.MAXIMIZE)

# Contraintes

# C1: Chaque élément "Contre" (j) doit être couvert exactement une fois, pour former 
# l'ensemble cons(x,y) à la fin
# sum_{i in P} x_ij == 1  pour tout j in C
for j in C_indices:
    # On somme sur tous les i qui forment une paire valide avec ce j
    m.addConstr(gp.quicksum(x[i, j] for i in P_indices if (i, j) in valid_pairs) == 1, name=f"cover_con_{matieres[j]}")

# C2: Chaque élément "Pour" (i) ne peut être utilisé qu'une seule fois au maximum
# sum_{j in C} x_ij <= 1 pour tout i in P
for i in P_indices:
    # On somme sur tous les j qui forment une paire valide avec ce i
    m.addConstr(gp.quicksum(x[i, j] for j in C_indices if (i, j) in valid_pairs) <= 1, name=f"use_pro_{matieres[i]}")

# 5. Résolution
m.optimize()

Set parameter Username
Set parameter LicenseID to value 2755075
Academic license - for non-commercial use only - expires 2026-12-15
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6 rows, 6 columns and 12 nonzeros
Model fingerprint: 0xa9562452
Variable types: 0 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 6 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 3 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%


In [4]:
# 6. Interprétation des résultats
nb_cons_total = len(C_indices)
nb_cons_covered = m.objVal

print("-" * 30)
print(f"Objectif atteint : {int(nb_cons_covered)} / {nb_cons_total} critères 'Contre' couverts.")

if nb_cons_covered == nb_cons_total:
    print("\nSUCCÈS : Une explication complète de type (1-1) a été trouvée !")
    print("Voici le raisonnement généré[cite: 17, 34]:")
    
    explanation_pairs = []
    for i, j in valid_pairs:
        if x[i, j].X > 0.5: # Si la variable est à 1
            print(f" -> L'avantage en {matieres[i]} (+{contributions[i]}) compense le désavantage en {matieres[j]} ({contributions[j]}).")
            explanation_pairs.append((matieres[i], matieres[j]))
            
    # Vérification par rapport à l'exemple du texte : {(A,C), (D,F), (E,G)} [cite: 35]
    print(f"\nPaires retenues : {explanation_pairs}")
else:
    print("\nÉCHEC : Impossible de fournir une explication complète avec des trade-offs (1-1).")
    print("Certains désavantages sont trop forts pour être compensés un-pour-un.")

------------------------------
Objectif atteint : 3 / 3 critères 'Contre' couverts.

SUCCÈS : Une explication complète de type (1-1) a été trouvée !
Voici le raisonnement généré[cite: 17, 34]:
 -> L'avantage en Anatomie (+32) compense le désavantage en Chirurgie (-28).
 -> L'avantage en Diagnostic (+36) compense le désavantage en Forensic Pathology (-35).
 -> L'avantage en Epidemiologie (+48) compense le désavantage en Génétique (-42).

Paires retenues : [('Anatomie', 'Chirurgie'), ('Diagnostic', 'Forensic Pathology'), ('Epidemiologie', 'Génétique')]
